In [1]:
import pandas as pd

In [2]:
triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'

In [3]:
from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import sklearn.ensemble as ske

In [4]:
song_df_1 = pd.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

In [5]:
song_df_1.head()

user_id             song_id  listen_count
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995             1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B             2
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0             1
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D             1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273             1

In [6]:
metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'

In [7]:
song_df_2 =  pd.read_csv(metadata_file)

In [8]:
song_merged_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

In [9]:
len(song_merged_df)

2000000

In [10]:
song_merged_df.head()

user_id             song_id  listen_count  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995             1   
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B             2   
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0             1   
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D             1   
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273             1   

             title                        release    artist_name  year  
0         The Cove             Thicker Than Water   Jack Johnson     0  
1  Entre Dos Aguas            Flamenco Para Niños  Paco De Lucia  1976  
2         Stronger                     Graduation     Kanye West  2007  
3   Constellations              In Between Dreams   Jack Johnson  2005  
4     Learn To Fly  There Is Nothing Left To Lose   Foo Fighters  1999

In [11]:
song_df = song_merged_df.head(10000)

In [12]:
song_df.head()

user_id             song_id  listen_count  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995             1   
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B             2   
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0             1   
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D             1   
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273             1   

             title                        release    artist_name  year  
0         The Cove             Thicker Than Water   Jack Johnson     0  
1  Entre Dos Aguas            Flamenco Para Niños  Paco De Lucia  1976  
2         Stronger                     Graduation     Kanye West  2007  
3   Constellations              In Between Dreams   Jack Johnson  2005  
4     Learn To Fly  There Is Nothing Left To Lose   Foo Fighters  1999

In [17]:
#merge title and artist into one column called song
pd.options.mode.chained_assignment = None  # default='warn'

In [18]:
song_df['song'] = song_df['title'].map(str) + '-' + song_df['artist_name']

In [19]:
song_df.head()

user_id             song_id  listen_count  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995             1   
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B             2   
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0             1   
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D             1   
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273             1   

             title                        release    artist_name  year  \
0         The Cove             Thicker Than Water   Jack Johnson     0   
1  Entre Dos Aguas            Flamenco Para Niños  Paco De Lucia  1976   
2         Stronger                     Graduation     Kanye West  2007   
3   Constellations              In Between Dreams   Jack Johnson  2005   
4     Learn To Fly  There Is Nothing Left To Lose   Foo Fighters  1999   

                            song  
0          The Cove-Jack Johnson  
1  Entre Dos Aguas-Paco De Lucia  
2            Stronger-Kanye West  
3    Constellations-Jack Johnson  
4      Learn To Fly-Foo Fighters

In [20]:
#group by song and take count of how many times the song was listened to by users
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()

In [21]:
#take sum of each song's listen count
lc_grouped_sum = song_grouped['listen_count'].sum()

In [22]:
song_grouped['percentage']  = song_grouped['listen_count'].div(lc_grouped_sum)*100

In [23]:
#songs by popularity
song_grouped.sort_values(['listen_count', 'song'], ascending = [0,1])

song  listen_count  \
3660                             Sehr kosmisch-Harmonia            45   
4678                                         Undo-Björk            32   
5105                       You're The One-Dwight Yoakam            32   
1071  Dog Days Are Over (Radio Edit)-Florence + The ...            28   
3655                                Secrets-OneRepublic            28   
4378                             The Scientist-Coldplay            27   
4712                         Use Somebody-Kings Of Leon            27   
3476                              Revelry-Kings Of Leon            26   
1387                       Fireflies-Charttraxx Karaoke            24   
1862  Horn Concerto No. 4 in E flat K495: II. Romanc...            23   
1805                             Hey_ Soul Sister-Train            22   
5032                                    Yellow-Coldplay            22   
808                                     Clocks-Coldplay            21   
2620  Lucky (Album Version)-Jason Mraz & Colbie Caillat            20   
2299               Just Dance-Lady GaGa / Colby O'Donis            19   
456   Billionaire [feat. Bruno Mars]  (Explicit Albu...            18   
2689                                     Marry Me-Train            18   
3064                      OMG-Usher featuring will.i.am            18   
4543                                   Tive Sim-Cartola            18   
142                                 Alejandro-Lady GaGa            17   
726   Catch You Baby (Steve Pitron & Max Sanna Radio...            17   
1410                              Float On-Modest Mouse            17   
3868                     Somebody To Love-Justin Bieber            17   
631                                 Bulletproof-La Roux            16   
1142                  Drop The World-Lil Wayne / Eminem            16   
3038  Nothin' On You [feat. Bruno Mars] (Album Versi...            16   
4465                  They Might Follow You-Tiny Vipers            16   
870                  Cosmic Love-Florence + The Machine            15   
899                          Creep (Explicit)-Radiohead            15   
1680                                       Halo-Beyoncé            15   
...                                                 ...           ...   
5094   You Yourself are Too Serious-The Mercury Program             1   
5098  You'll Never Know (My Love) (Bovellian 07 Mix)...             1   
5100                     You're A Wolf (Album)-Sea Wolf             1   
5102   You're Gonna Miss Me When I'm Gone-Brooks & Dunn             1   
5103                               You're Not Alone-ATB             1   
5104                             You're Not Alone-Olive             1   
5108                   You've Passed-Neutral Milk Hotel             1   
5110                             Young-Hollywood Undead             1   
5111            Younger Than Springtime-William Tabbert             1   
5112              Your Arms Feel Like home-3 Doors Down             1   
5113                   Your Every Idol-Telefon Tel Aviv             1   
5114        Your Ex-Lover Is Dead (Album Version)-Stars             1   
5115     Your Guardian Angel-The Red Jumpsuit Apparatus             1   
5117                         Your House-Jimmy Eat World             1   
5119                             Your Love-The Outfield             1   
5121                        Your Mouth-Telefon Tel Aviv             1   
5123          Your Song (Alternate Take 10)-Cilla Black             1   
5126         Your Visits Are Getting Shorter-Bloc Party             1   
5127                              Your Woman-White Town             1   
5130                     Ze Rook Naar Rozen-Rob De Nijs             1   
5131                                  Zebra-Beach House             1   
5132                                      Zebra-Man Man             1   
5133                            Zero-The Pain Machinery             1   
5135               Zopf: Pigtail-Pen

In [24]:
users = song_df['user_id'].unique()
songs = song_df['song_id'].unique()


In [26]:
len(users)

365

In [27]:
len(songs)

5175